Library import
dotenv

In [2]:
import os
from dotenv import load_dotenv
import requests as rq
import pandas as pd
import io
import json
load_dotenv()

True

In [5]:
def load_json_file(file_path):  # Function to load and parse a JSON file in read mode
    with open(file_path, 'r') as file:
        return json.load(file)  # Parse the JSON content and return the resulting Python data structure


def find_title_by_id(dictionaryJason, search_id):  # Function to find a title by its ID if  'dictionaryJason' is a dictionary where keys are IDs and values are titles
    return dictionaryJason.get(search_id, None) # Return the value associated with search_id if it exists, otherwise return None


def process_string(input_string):   # Function to process a string according to specific rules (sobsitute space with _ and clear the end)
    if input_string:     # Check if the input string is not empty or None 
        processed_string = input_string.replace(' ', '_')   # Replace all spaces with underscores
        processed_string = processed_string.split('-')[0]    # Split the string at the first hyphen and take only the first part
        
        
        if processed_string.endswith('_'):  # Remove trailing underscore if present
            processed_string = processed_string[:-1]
            
        return processed_string
    return  # Return None if the input string is empty or None

def truncate_string(input_string, max_length):  # Function to truncate a string to a maximum length, cutting at the last underscore
    
    if len(input_string) > max_length:  # Check if the string is longer than the maximum allowed length
       
        last_underscore_index = input_string.rfind('_', 0, max_length)  # Find the position of the last underscore before the maximum length
        
        
        if last_underscore_index != -1: # If an underscore is found, truncate the string at that position
            return input_string[:last_underscore_index]
    return input_string # Return the original string if it's not longer than max_length or if no underscore is found before max_length

variabili per decidere il dataflow da scaricare e il tipo di file della risposta

In [7]:
datatype = os.getenv('datatype')
dataflow = os.getenv('dataflow')
filter = os.getenv('filter')
timeframe =  os.getenv('timeframe')
tableNamePath = 'istat_metadata_extractor\\extracted\\122_54_metadata\\dataflows.json'
url = f'https://esploradati.istat.it/SDMXWS/rest/data/{dataflow}/{filter}?{timeframe}'
headersCsv = {'Accept':'text/csv'}
headersJson = {'Accept':'application/json'}
data = load_json_file(tableNamePath)
tableString = find_title_by_id(data, dataflow)
tempTableName = process_string(tableString)
if tempTableName:
    if len(tempTableName)<63:
        tableName = tempTableName
    else:
        print(tempTableName)
        tableName=truncate_string(tempTableName,55)
else:
    print(f"Nessun titolo trovato per l'id {dataflow}")
    tableName = dataflow
print(tableName)
print(url)

Nessun titolo trovato per l'id None
None
https://esploradati.istat.it/SDMXWS/rest/data/None/None?None


In [ ]:
if datatype == 0:
    response = rq.get(url,headers=headersCsv)
    if response.status_code == 200:
        #print(response.text[:500])  # debug missed Df print
        try:
            Df = pd.read_csv(io.StringIO(response.text))
            #print('from CSV')  #debug esaminare Df
            #display(Df)        #debug esaminare Df
        except Exception as e:
            print(f"Error parsing CSV: {e}")
    else:
        print(f"Error: Status code {response.status_code}")
        print(response.text)
else:
    if datatype == 1:
        response = rq.get(url,headers=headersJson)
        if response.status_code == 200:
            try:
                Df = pd.json_normalize(response.json())
                #print('from JSON') #debug esaminare Df
                #display(Df)        #debug esaminare Df
            except Exception as e:
                print(f"Error parsing json: {e}")
        else:
            print(f"Error: Status code {response.status_code}")
            print(response.text)


Drop columns where all values are NaN

In [ ]:
Df = Df.dropna(axis=1, how='all')
display(Df)

definition of the connection string of the DB

In [1]:
def get_connection_string():
    """Crea una stringa di connessione."""
    return f"postgresql://{os.getenv('user')}:{os.getenv('password')}@{os.getenv('host')}:{os.getenv('port')}/{os.getenv('database')}"

In [56]:
Df.to_sql(tableName, get_connection_string(), if_exists='replace', index=False)

80